# Example output plots <a class="anchor" id="top"></a>


### Example run
The example data used to make these plots is currently in the same directory as the Jupyter notebook - you will want to change this to where you have saved your results.
The run shown here is for a 500km planetesimal, accreted 0.8Ma after CAI formation, $X_{S,0}$=26.7 wt%, all other parameters are given by run 1 in auto_params.csv.

+ [Summary stats](#stats)
+ [Temperature profile](#temp)
+ [Heat fluxes](#flux)
+ [Field strength and Reynolds number](#Bfield)

### Set-up

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sys
sys.path.append('../')
from load_info import load_run_info, load_run_results

#scale time to Myr
from plot_params import Myr, f0

Choose run for main thermal plots and where to save files.

In [ ]:
run=12
save = False # do you want to save your figures?
automated = False
log_time = True #do you want to plot time logarithmically
path = '' #path to files 
save_path = '../Plots/'

Load results for differentiation

In [ ]:
npzfile = np.load(f'{path}run_{run}_diff.npz')
Tdiff = npzfile['Tdiff'] 
iron = npzfile['Xfe']
silicate_d = npzfile['Xsi']
tdiff = npzfile['t_diff']
Ra_d = npzfile['Ra']
Ra_crit_d = npzfile['Ra_crit']
d0_diff = npzfile['d0']

Load results for thermal evolution

In [ ]:
npzfile = np.load(f'{path}run_{run}.npz')
Tc= npzfile['Tc'] 
Tc_conv = npzfile['Tc_conv']
Tcmb = npzfile['Tcmb']
Tm_mid = npzfile['Tm_mid']
Tm_conv = npzfile['Tm_conv']
Tm_surf = npzfile['Tm_surf'] 
T_profile = npzfile['T_profile']
f = npzfile['f'] 

t = npzfile['t'] #time in s
Rem = npzfile['Rem'] # magnetic Reynolds number 
B = npzfile['B']/1e-6 # magnetic field strength [T] 
buoyr = npzfile['buoyr'] #compositional buoyancy flux and thermal buoyancy flux
Flux = npzfile['Flux']
Ra = npzfile['Ra'] 
RaH = npzfile['RaH'] 
RanoH = npzfile['RanoH'] 
Racrit = npzfile['Racrit'] 
d0 = npzfile['d0'] 
min_unstable = npzfile['min_unstable'] 
Urey = npzfile['Ur']
Xs = npzfile['Xs']
dl = npzfile['dl']
dc = npzfile['dc']
Fs = Flux[0]
Fcmb = Flux[1]
Fad = Flux[2]
Frad = Flux[3]

Concatenate shared variables

In [ ]:
Tall = np.vstack((Tdiff,T_profile))
tall = np.append(tdiff,t)
iron_all = np.hstack
Ra_all = np.append(Ra_d,Ra)
Ra_crit_all = np.append(Ra_crit_d,Racrit)
d0_all = np.append(d0_diff,d0)

Scale time by Myr

In [ ]:
t_plot_all = tall/Myr
t_plot_t = t/Myr

Run info

In [ ]:
if automated == True:
    r, rcr, dr, tstart, dt, viscosity, icfrac = load_run_info(run,f'{path}auto_params.csv')
else: 
    r, rcr, dr, tstart, dt, viscosity, icfrac = load_run_info(run,f'{path}run_info.csv')

## Summary statistics <a class="anchor" id="stats"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Load in results data

In [ ]:
results = load_run_results(run,f'{path}run_results.csv')

Load in run parameters

In [ ]:
if automated == True:
    params = pd.read_csv(f'{path}auto_params.csv',skiprows=[1])
    params = params[params['run']==run]
else:
    params = pd.read_csv(f'{path}run_info.csv',skiprows=[1])
    params = params[params['run']==run]

Assign to variables values that will be used later

In [ ]:
n_cells = int(r/dr)
nccells = round((n_cells-3)*(rcr))+2 #number of cells needed to span core (inc. centre and CMB)
nmantle = n_cells - nccells +1 #number of cells needed to span mantle plus one extra for CMB
rc = (nccells-1)*dr #radius of core [m], subtract one for centre
terode = results.at[0,'terode']
tstrat_remove = results.at[0,'tstrat_remove']
diff_time = results.at[0,'diff_time']
peak_coreT = np.amax(Tall[:,:nmantle])
loc_max = np.where(Tall[:,:nmantle]==peak_coreT)[1][0] #take the set of time coordinates and first value (they should all be the same)
tcoremax = tall[loc_max]/Myr
fcond_t = results.at[0,'fcond_t']
diff_T =results.at[0,'diff_T']

#dynamo on and off times
var_results = pd.read_csv(path+'run_results.csv',skiprows=[1])
on1=var_results.loc[var_results['run']==run,'magon_1'].values[0]
off1=var_results.loc[var_results['run']==run,'magoff_1'].values[0]
on2=var_results.loc[var_results['run']==run,'magon_2'].values[0]
off2=var_results.loc[var_results['run']==run,'magoff_2'].values[0]
on3=var_results.loc[var_results['run']==run,'magon_3'].values[0]
off3=var_results.loc[var_results['run']==run,'magoff_3'].values[0]

Print a summary

In [ ]:
print(f"Differentiation is at {results.at[0,'diff_time']:.2f} Myr")
print(f"The temperature at differentiation is at {results.at[0,'diff_T']:.2f}K")
print(f"Peak magma ocean temp is {results.at[0,'peakT']:.0f}K at {results.at[0,'tmax']:.2f} Myr")
print(f"Stratification starts at {results.at[0,'tstrat_start']:.2f} Myr")
print(f"Mantle hotter than the core until {results.at[0,'tstrat_remove']:.2f} Myr")
print(f"Erosion of stratification by {results.at[0,'terode']:.2f} Myr") 
print(f"End of mantle convection by {results.at[0,'fcond_t']:.2f} Myr")
print(f"The maximum thermal Rem is {results.at[0,'max_Rtherm']:.2f} at {results.at[0,'max_Rtherm']:.2f} Myr")
print(f"The maximum thermal field strength is {results.at[0,'max_Btherm']:.2e}T at {results.at[0,'max_Bthermt']:.2f} Myr")
print(f"The maximum compositional Rem is {results.at[0,'max_Rcomp']:.2f}")
print(f"The maximum compositional field strength is {results.at[0,'max_Bcomp']:.2e}")
print(f'The dynamo starts at {on1:.2f} Myr, stops at {off1:.2f} Myr and lasts {off1-on1:.2f} Myr')
if on2 > 0:
    print(f'The second dynamo starts at {on2:.2f} Myr, stops at {off2:.2f} Myr and lasts {off2-on2:.2f} Myr')
if on3 > 0:
    print(f'The third dynamo starts at {on3:.2f} Myr, stops at {off3:.2f} Myr and lasts {off3-on3:.2f} Myr')
print(f"Core solidification begins at {results.at[0,'tsolid_start']:.2f} Ma and ends at {results.at[0,'tsolid']:.2f} Ma")

# Example run

## Temperature profile <a class="anchor" id="temp"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Create r array for plotting and prelog time to speed up plotting

In [ ]:
rplot = np.arange(0,int(r)+dr,int(dr))/1e3
r_unstable=np.array([]) 
for ind in min_unstable:
    r_unstable = np.append(r_unstable,rplot[int(ind)])


#log time
if log_time == True:
    tpt = np.log10(t_plot_t)
    tpa = np.log10(t_plot_all)
    lfcond = np.log10(fcond_t)
    on1l = np.log10(on1)
    on2l = np.log10(on2)
    on3l = np.log10(on3)
    off1l = np.log10(off1)
    off2l = np.log10(off3)
    off3l = np.log10(off3)
else: 
    tpt = t_plot_t
    tpa = t_plot_all
    lfcond = fcond_t
    on1l = on1
    on2l = on2
    on3l = on3
    off1l = off1
    off2l = off2
    off3l = off3

Make figure

In [ ]:
plt.figure(figsize=[10,20/3])
plt.pcolormesh(tpa[::2],rplot[::2],np.transpose(Tall[::2,::2]),shading = 'gouraud',vmin=200,vmax=1600) 
plt.hlines(rc/1e3,tpt[0],max(tpa),linestyle='--',color='black',label='CMB')
plt.vlines(tpt[0],0,r/1e3,linestyle='-.',label='Differentiation')
plt.fill_betweenx([0,rc/5e3],on1l,off1l,alpha=0,hatch='/',label='dynamo on')
plt.plot(tpt,r_unstable,linestyle='dotted',label='Convecting core')
if on2 > 0:
    plt.fill_betweenx([0,rc/5e3],on2l,off2l,alpha=0,hatch='/')
if on3 > 0: 
    plt.fill_betweenx([0,rc/5e3],on3l,off3l,alpha=0,hatch='/')

if np.any(t_plot_t<fcond_t):
    plt.plot(tpt[(t_plot_t<=fcond_t)&(d0<(r-rc))],(r-d0[(t_plot_t<=fcond_t)&(d0<(r-rc))])/1e3,linestyle='dashed',label='base of $\delta_0$',color='blue')
    plt.plot(tpt[(t_plot_t<=fcond_t)&(r_unstable==0)][1:],(rc+dl[(t_plot_t<=fcond_t)&(r_unstable==0)][1:])/1e3,linestyle='dotted',label='top of $\delta_l$',color='blue')
    plt.vlines(tpt[t_plot_t<=fcond_t][-1],r/1e3,rc/1e3,linestyle='dotted',label='conductive mantle',color='red')
plt.plot(tpt[f<f0],f[f<f0]*rc/1e3,linestyle='-.',color='black',label='Top of liquid core')
#labels and limits
plt.ylabel('Distance from centre of planetesimal/km')
if log_time == False:
    plt.xlabel('Time / Ma')
else:
    plt.xlabel('log10(Time / Ma)')
plt.colorbar(label='Temperature/K')
plt.ylim(bottom=0)
plt.legend(bbox_to_anchor=[1.6,0.5])
#plt.xscale('log')
if save == True:
    plt.savefig(f'{save_path}run_{run}_thermal_profile.png',bbox_inches='tight',dpi=500)

How long is the planetesimal below 700K?

In [ ]:
tacc = 0.8 #accretion time in Ma
rlen = int((r/1e3)*2)
tsint = np.zeros([rlen])
for i in range(rlen):
    tsint[i]=tall[Tall[:,i]>700][0]
tshort = np.min(tsint/Myr-tacc)
print(f'The shortest time before sintering is {tshort:.2f} Ma \
and the longest time before sintering is {np.max(tsint/Myr-tacc):.2f} Ma')

In [ ]:
rcheck = rplot[:-1] #remove surface point
rsint = rcheck[(tsint/Myr)>(tshort+tacc)]
print(f'The inner {rsint[0]}km sinters within {tshort:.1f} Ma of accretion')

## Temperature and heat fluxes <a class="anchor" id="flux"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Have plotted only the central core temperature and convective mantle temperature.

In [ ]:
dl_end = int(dl[Tm_conv==0][0]/dr) #index of CMB b.l. base above CMB at cessation of convection

with sns.plotting_context('paper',font_scale=1.4):
    plt.figure(tight_layout=True,figsize=[10,7])
    xmin=tstart

    #temperatures as function of time
    plt.subplot(2,1,1)

    plt.plot(t_plot_t,Tc,label='central core temperature',color='black')
    plt.plot(t_plot_t[Tm_conv!=0],Tm_conv[Tm_conv!=0],label='convective mantle temperature',color='#FF5350')
    plt.plot(t_plot_t[Tm_conv==0],T_profile[Tm_conv==0,nmantle+dl_end+2],label='conductive mantle temperature',color='#FF5350',linestyle='dashed')
    if log_time == True:
        plt.xscale('log')
    plt.ylim([1250,1550])
    plt.ylabel('T/K')
    plt.legend(loc='lower left')

    #fluxes as function of time
    plt.subplot(2,1,2)
    Fcmb_neg = Fcmb[Fcmb<0]
    Fcmb_pos = Fcmb[Fcmb>0]

    plt.semilogy(t_plot_t,Fs,label='$F_s$',color='#0F4C5C')
    plt.scatter(t_plot_t[Fcmb<0],abs(Fcmb_neg),label='$-F_{CMB}$',color='#5BC0EB',s=2)
    plt.scatter(t_plot_t[Fcmb>0],Fcmb_pos,label='$F_{CMB}$',color='#5F0F40',s=2)
    plt.semilogy(t_plot_t,Fad,label='$F_{ad}$',color='#E36414',linestyle='dashed')
    plt.semilogy(t_plot_t,Frad,label='$F_{rad}$',color='#9A031E',linestyle='-.')
    plt.fill_betweenx(y=[1e-4,100],x1=tstrat_remove,x2=terode,label='erosion of stratification',alpha=0.2)
    plt.fill_betweenx(y=[1e-4,100],x1=fcond_t,x2=max(t_plot_t),label='mantle conducting',alpha=0.2)
    plt.fill_betweenx(y=[1e-4,1e-3],x1=on1,x2=off1,label='dynamo on',alpha=0.2,color='grey')
    plt.fill_betweenx(y=[1e-4,1e-3],x1=on2,x2=off2,alpha=0.2,color='grey')
    if log_time == True:
        plt.xscale('log')
    plt.xlabel('Time/ Ma')

    plt.ylim([1e-4,1e2])   #use these limits when comparing runs
    plt.ylabel('Heat flux/ W$m^{-2}$')
    plt.legend(loc='upper right',ncol=2)

    if save == True:
        plt.savefig(f'{save_path}run_{run}_flux.png',dpi=500)

## Magnetic Field Strength <a class="anchor" id="Bfield"></a>
<p align="right">(<a href="#top">back to top</a>)</p>

Extract compositional and thermal buoyancy components

In [ ]:
comp = buoyr[0,:]
therm = buoyr[1,:]

In [ ]:
Xs_eutectic = 33
print(f'The core reaches the eutectic at {t_plot_t[Xs>=Xs_eutectic][0]:.2f} Ma')

Create data frames for rolling average compositional dynamo strengths

In [ ]:
compdf = pd.Series(comp[f<f0])
thermdf = pd.Series(therm[f<f0])
Remdf = pd.Series(Rem[f<f0])
Bdf = pd.Series(B[f<f0])

Make figure

There is a lag between the onset of solidification and the rolling average so plot the original time series and the average.

In [ ]:
wn = 75 #averaging window width
fig, ax = plt.subplots(nrows=2,ncols=1,sharex='col',figsize=[10,8])
#B and Rem
ax2 = ax[0].twinx()
ln1 = ax[0].plot(t_plot_t[f>=f0],np.ma.masked_where(Rem[f>=f0]<10,B[f>=f0]),color='black')
ax[0].plot(t_plot_t[f<f0],np.ma.masked_where(Rem[f<f0]<10,B[f<f0]),color='black',alpha=0.1) #original series
ax[0].plot(t_plot_t[f<f0],Bdf.rolling(window=wn,center=True).mean(),color='black') #rolling average
ln2 = ax2.plot(t_plot_t[f>=f0],Rem[f>=f0],color='darkorchid')
ax2.plot(t_plot_t[f<f0],Rem[f<f0],color='darkorchid',alpha=0.1) #original series
ax2.plot(t_plot_t[f<f0],Remdf.rolling(window=wn,center=True).mean(),color='darkorchid') #rolling average


ax2.hlines(10,min(t_plot_t),max(t_plot_t),linestyle='dashed',color='grey')
ax[0].legend(ln1+ln2,['Magnetic field strength','Re$_m$'])
ax[0].set_ylabel('Magnetic field strength/ $\mu T$')
ax2.set_ylabel('$Re_m$')
ax[0].tick_params(axis='y',colors='black')
ax2.tick_params(axis='y',colors='darkorchid')
ax[0].yaxis.label.set_color('black') 
ax2.yaxis.label.set_color('darkorchid') 

#thermal and compositional buoyancy flux
ln3 = ax[1].plot(t_plot_t[f>=f0],therm[f>=f0],color='black')
#ax[1].plot(t_plot_t[f<f0],therm[f<f0],color='black',alpha=0.1)
ax[1].plot(t_plot_t[f<f0],thermdf.rolling(window=wn,center=True).mean(),color='black')
ax[1].plot(t_plot_t[f<f0],comp[f<f0],color='darkorchid',alpha=0.1)
ln4=ax[1].plot(t_plot_t[f<f0],compdf.rolling(window=wn,center=True).mean(),color='darkorchid')
ax[1].legend(ln3+ln4,['thermal','compositional'])
ax[1].set(ylim=[1e-11,1e-8],ylabel='Buoyancy flux per unit area /kg$s^{-1}m^{-2}$',xlabel='Time/Ma',xscale='log')

if save == True:
    plt.savefig(f'{save_path}run_{run}_Bbuoy.png',dpi=500)

For the 500km body the buoyancy flux ratio is not useful, because of the cessation of mantle convection in the middle, but can add the code below to the plot abovr if you want to plot it.